# Baseline Neural Network Model Performance

In [0]:
import numpy as np
import pandas as pd
from tensorflow.keras import models
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [0]:
# load dataset
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [46]:
from sklearn.preprocessing import LabelEncoder
label_encoder_op = LabelEncoder()
encoded_Y=label_encoder_op.fit_transform(Y)
encoded_Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [5]:
X.shape

(208, 60)

In [0]:
encoded_Y=label_encoder_op.fit_transform(Y)

In [0]:
# baseline model
def create_baseline():
	# create model, write code below
  model=models.Sequential()
  model.add(layers.Dense(60,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(1, activation='sigmoid'))
  # Compile model, write code below
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [8]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Results: 81.73% (7.05%)


# Baseline Model With Data Preparation (Pipeline)

In [9]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 83.16% (6.14%)


# Evaluate a Smaller Network

In [10]:
# smaller model
def create_smaller():
	# create model
  model=models.Sequential()
  model.add(layers.Dense(30,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(1, activation='sigmoid'))
	# Compile model
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 84.59% (4.30%)


# Evaluate a Larger Network

In [11]:
# larger model
def create_larger():
	# create model
  model=models.Sequential()
  model.add(layers.Dense(60,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(30, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
	# Compile model
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 86.04% (4.52%)


# Scaling up: Developing a model that overfits

In [14]:
def create_overfit():
	# create model
  model=models.Sequential()
  model.add(layers.Dense(200,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(160, activation='relu'))
  model.add(layers.Dense(130, activation='relu'))
  model.add(layers.Dense(50, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
	# Compile model
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_overfit, epochs=250, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Overfit: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Overfit: 83.56% (5.66%)


# Tuning the Model

In [17]:
def create_best():
	# create model
  model=models.Sequential()
  model.add(layers.Dense(60,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(30, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
	# Compile model
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_best, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Optimized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Train on 186 samples
Epoch 1/100
186/186 [==============================] - 1s 4ms/sample - loss: 0.6191 - acc: 0.6183
Epoch 2/100
186/186 [==============================] - 0s 686us/sample - loss: 0.4283 - acc: 0.8226
Epoch 3/100
186/186 [==============================] - 0s 739us/sample - loss: 0.3217 - acc: 0.9032
Epoch 4/100
186/186 [==============================] - 0s 743us/sample - loss: 0.2406 - acc: 0.9247
Epoch 5/100
186/186 [==============================] - 0s 811us/sample - loss: 0.1776 - acc: 0.9677
Epoch 6/100
186/186 [==============================] - 0s 764us/sample - loss: 0.1331 - acc: 0.9839
Epoch 7/100
186/186 [==============================] - 0s 771us/sample - loss: 0.0991 - acc: 0.9892
Epoch 8/100
186/186 [==============================] - 0s 740us/sample - loss: 0.0721 - acc: 0.9946
Epoch 9/100
186/186 [==============================] - 0s 771us/sample - loss: 0.0497 - acc: 0.9946
Epoch 10/100
186/186 [==============================] - 0s 715us/sample - loss: 0

# Rewriting the code without using scikit-learn

In [49]:
encode_Y=pd.get_dummies(Y)
# create model, write code below
def build_model():
  model=models.Sequential()
  model.add(layers.Dense(60,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(1, activation='sigmoid'))
# Compile model, write code below
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model
k = 10
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []
for i in range(k):
  print('processing fold #', i)
  val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate(
  [X[:i * num_val_samples],
  X[(i + 1) * num_val_samples:]],
  axis=0)
  partial_train_targets = np.concatenate(
  [encoded_Y[:i * num_val_samples],
  encoded_Y[(i + 1) * num_val_samples:]],
  axis=0)
  model = build_model()
  model.fit(partial_train_data, partial_train_targets,
  epochs=num_epochs, batch_size=1, verbose=0)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [50]:
all_scores

[0.4, 0.65, 0.5, 0.95, 0.35, 0.25, 0.7, 0.6, 0.5, 1.0]

In [51]:
np.mean(all_scores)

0.59000003